In [1]:
# Enable automatic reloading of modules before executing code
%load_ext autoreload
%autoreload 2

In [118]:
import os
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [3]:
import utils as ut

In [19]:
# test run of the model for dev purposes, or full/final data set?
is_testcase = True

# Load Model Results from Disk

In [4]:
l_lmbda = [0.0005]
l_lmbda_hierarchical = [0.01]
l_rnd_seed = [7640]
l_embed_dim = [7]
l_sparse = ["items_and_random_ids"]
modeltype = "random_weights_random_scaling"
modelversion = "embeddings-decision-combined-data"

In [25]:
# load results from model run
l_results = load_ID_lowdim(
    l_lmbda=l_lmbda, l_lmbda_hierarchical=l_lmbda_hierarchical, l_rnd_seed=l_rnd_seed, 
    l_embed_dim=l_embed_dim, l_sparse=l_sparse, modelversion=modelversion, modeltype=modeltype
)

In [39]:
# Load Relevant IDs for correlational analyses
if is_testcase:
    df_pids = pd.read_csv("data/study1-2025-08/new-participant-ids-in-joint-modeling-testcase.csv")
elif not is_testcase:
    df_pids = pd.read_csv("data/study1-2025-08/new-participant-ids-in-joint-modeling.csv")

In [52]:
l_pids_model = list(df_pids["participant_id_model"])
l_pids_new = list(df_pids["participant_id_new"])

In [53]:
df_dim_weights = pd.DataFrame(l_results[0]["decision_weights"].detach().numpy()[l_pids_model, :])
# participant_id_new can be joined with results coming only from the new study
df_dim_weights["participant_id_new"] = l_pids_new
# Move participant_id column to the first position
col = "participant_id_new"
new_order = [col] + [c for c in df_dim_weights.columns if c != col]
df_dim_weights = df_dim_weights[new_order]
df_dim_weights

,participant_id_new,0,1,2,3,4,5,6
0,1,1.201567,0.598141,0.626638,1.406600,0.127356,0.372309,0.363178
1,2,1.154933,0.331691,1.248743,0.674811,1.291103,0.332149,0.592179
2,3,0.552247,0.333345,1.197031,0.575579,0.993515,0.359048,0.580531
3,4,0.791544,0.632017,1.233657,1.050458,1.188171,0.723804,0.440776


In [151]:
# load dfs saved with R
df_qs_num_long = pd.read_csv("data/study1-2025-08/tbl_qs_num_long_excluded.csv")
df_qs_txt = pd.read_csv("data/study1-2025-08/tbl_qs_txt_excluded.csv")
df_qs_num = df_qs_num_long.pivot(index=["participant_id", "session_id"], columns="name", values="value").reset_index(drop=False)
# reverse code items
df_qs_num = ut.reverse_code_q_items(df_qs_num)
# create scale and facet scores
df_qs_num = ut.scales_and_facets(df_qs_num)

*todos*
- create text embeddings for interest questions
- merge with dim weights
- create features with dim weights:
    - average dim weight
    - gini coef
- run model with split-half data
- load results and create feature "stability" of weights between halfs

In [55]:
df_qs_txt.head()

,participant_id,session_id,workHistory,interests1,interests2,interests3,feedback
0,1,1,Barista,"Sport results, trading card games, video games","Playing trading card games, board games, video...",Biomedical Science,"No further feedback, study was easy to underst..."
1,2,1,IT specialist for system integration,"politics, gaming, veganism, music","playing video games, watching shows, meeting w...","philosophy, astronomy",Was pretty interesting but it would've been ni...
2,3,1,"Storeman, Inventory Controller, Buyer, Senior ...","Sport, Football, Politics",Watching local football team,"Increasing global inequality,",Nothing to add
3,5,1,"Nursing Assistant, Nurse","Entertainment, Politics, Medicine & Healthcare...","Watching series, Playing video games, Listenin...","Computer programing, Data Analytics",NaN
